In [35]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from geopack import geopack

Load IGRF coefficients ...


In [45]:
#open all .txt files in a directory in pandas

def open_files(directory):
    files = os.listdir(directory)
    dataframes = []
    for file in files:
        if file.endswith('.txta'):

            data = pd.read_csv(directory+file, sep='\s+', header=None)
            dataframes.append(data)
        
            df = pd.concat(dataframes)

    df['dt'] = pd.to_datetime((df[0].astype(str) + df[1].astype(str) + df[2].astype(str)), format='%Y%m%d%H%M')
    
    #drop first 3 columns and move last column to first
    df = df.drop([0,1,2,7,8,9,10], axis=1)
    cols = list(df.columns)
    cols = [cols[-1]] + cols[:-1]
    df = df[cols]

    df = df.sort_values(by='dt')  
    df = df.reset_index(drop=True)

    df_labs = ['dt', 'np', 'vsw', 'By', 'Bz']
    df.columns = df_labs
    
    #apply constants to columns
    df['np'] = df['np'] * 10 #in cm^-3
    df['vsw'] = (df['vsw'] * 5.32e4) / 1000 #in km/s
    df['By'] = df['By'] * 10 #in nT
    df['Bz'] = df['Bz'] * 10    #in nT

    return df

#open all .txt files in a directory in xarray
sw_df = open_files('/run/media/sachin/0fa21ddb-f70c-4238-9cf4-705e0360f1c1/NICT DUMP/sw2020_2024/')
#sw_df = open_files('/run/media/sachin/0fa21ddb-f70c-4238-9cf4-705e0360f1c1/Test_SW/')


sw_df

<>:9: SyntaxWarning: invalid escape sequence '\s'
<>:9: SyntaxWarning: invalid escape sequence '\s'
/tmp/ipykernel_7537/1100088584.py:9: SyntaxWarning: invalid escape sequence '\s'
  data = pd.read_csv(directory+file, sep='\s+', header=None)


,dt,np,vsw,By,Bz
0,2020-12-29 00:00:00,2.44,404.8520,-3.3,0.87
1,2020-12-29 00:01:00,2.48,402.9900,-4.2,0.34
2,2020-12-29 00:02:00,2.48,402.9900,-4.1,0.57
3,2020-12-29 00:03:00,2.48,402.9900,-4.4,-0.61
4,2020-12-29 00:04:00,2.48,402.9900,-4.3,-0.64
...,...,...,...,...,...
1800814,2024-08-31 23:52:00,2.40,399.5320,7.1,-6.38
1800815,2024-08-31 23:53:00,1.60,394.3716,7.2,-5.92
1800816,2024-08-31 23:54:00,2.14,387.1364,7.6,-4.93
1800817,2024-08-31 23:55:00,2.14,387.1364,9.2,-2.83


In [46]:
def calculate_tilt_angle(row):
    t0 = np.datetime64('1970-01-01T00:00:00')
    t1 = row['dt']
    tdiff = (t1 - t0).total_seconds()
    tilt_angle = geopack.recalc(tdiff) # Calculate dipole tilt angle
    
    return tilt_angle

sw_df['tilt_angle'] = sw_df.apply(calculate_tilt_angle, axis=1)


,dt,np,vsw,By,Bz,tilt_angle
0,2020-12-29 00:00:00,2.44,404.8520,-3.3,0.87,-0.446872
1,2020-12-29 00:01:00,2.48,402.9900,-4.2,0.34,-0.447563
2,2020-12-29 00:02:00,2.48,402.9900,-4.1,0.57,-0.448253
3,2020-12-29 00:03:00,2.48,402.9900,-4.4,-0.61,-0.448943
4,2020-12-29 00:04:00,2.48,402.9900,-4.3,-0.64,-0.449632
...,...,...,...,...,...,...
1800814,2024-08-31 23:52:00,2.40,399.5320,7.1,-6.38,0.098721
1800815,2024-08-31 23:53:00,1.60,394.3716,7.2,-5.92,0.098050
1800816,2024-08-31 23:54:00,2.14,387.1364,7.6,-4.93,0.097379
1800817,2024-08-31 23:55:00,2.14,387.1364,9.2,-2.83,0.096709


In [48]:
#resaample to 2 mins and interpolate over nan
sw_df = sw_df.resample('2min', on='dt').mean()
sw_df = sw_df.interpolate()
sw_df

,np,vsw,By,Bz,tilt_angle
dt,,,,,
2020-12-29 00:00:00,2.46,403.9210,-3.75,0.605,-0.447218
2020-12-29 00:02:00,2.48,402.9900,-4.25,-0.020,-0.448598
2020-12-29 00:04:00,2.48,402.9900,-3.75,0.280,-0.449976
2020-12-29 00:06:00,2.70,400.5960,-2.85,1.600,-0.451351
2020-12-29 00:08:00,2.70,400.5960,-2.90,2.360,-0.452723
...,...,...,...,...,...
2024-08-31 23:48:00,1.68,390.2220,6.25,-7.130,0.101079
2024-08-31 23:50:00,1.68,390.2220,5.95,-7.090,0.099731
2024-08-31 23:52:00,2.00,396.9518,7.15,-6.150,0.098385


In [52]:
export_path = f'/home/sachin/Documents/NIPR/Research/Data/OMNI/1-min/smrai3/' #server
df_name = f'rtsw_2min_2020-2024'
sw_df.to_csv(export_path + df_name + '.csv', index=True)